In [ ]:
# Since the cvxpy module is not available, we will provide a detailed guide for solving the problem in the user's own environment.

# Guide for solving Soft-Margin SVM using CVXPY with C = 1

import numpy as np
import cvxpy as cp
import pandas as pd

# Load the dataset, skip the first row
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/svm_train.txt', header=None, delim_whitespace=True, skiprows=1)

# The rest of your code remains unchanged
X = data.iloc[:, :-1].values  # Feature vectors
y = data.iloc[:, -1].values  # Labels


# Number of data points and features
n, d = X.shape

# Define the variables for CVXPY
w = cp.Variable(d)  # Weight vector
b = cp.Variable()   # Bias term
xi = cp.Variable(n) # Slack variables

# Regularization parameter
C = 1

# Formulate the objective function
objective = cp.Minimize(0.5 * cp.norm(w, 2)**2 + C * cp.sum(xi))

# Add the constraints
constraints = [cp.multiply(y, X @ w + b) >= 1 - xi, xi >= 0]

# Define and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Extract the optimal values
w_opt = w.value
b_opt = b.value

# Print the optimal w and b
print("Optimal w:", w_opt)
print("Optimal b:", b_opt)



Optimal w: [-1.00209063  3.64672864]
Optimal b: -1.8672371699699344


In [7]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/svm_train.txt', header=None, delim_whitespace=True, skiprows=1)
# Extracting features and labels from the data
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Number of data points
n = X.shape[0]

from cvxopt import matrix, solvers

# Defining the linear kernel function
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

# Function for Soft-Margin SVM Dual formation
def soft_margin_svm_dual(X, y, C):
    n_samples, n_features = X.shape

    # Calculating the Gram matrix
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            K[i, j] = linear_kernel(X[i], X[j])

    # Converting into cvxopt format
    P = matrix(np.outer(y, y) * K)
    q = matrix(-np.ones(n_samples))
    A = matrix(y, (1, n_samples), 'd')
    b = matrix(0.0)
    G = matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
    h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))

    # Solving the quadratic programming problem
    solvers.options['show_progress'] = False
    solution = solvers.qp(P, q, G, h, A, b)

    # Extracting the Lagrange multipliers
    alphas = np.ravel(solution['x'])

    # Support vectors have non zero lagrange multipliers
    sv = alphas > 1e-5
    ind = np.arange(len(alphas))[sv]
    alpha_sv = alphas[sv]
    sv_x = X[sv]
    sv_y = y[sv]

    # Calculating the intercept
    b = np.mean([sv_y[i] - np.sum(alpha_sv * sv_y * K[ind[i], sv]) for i in range(len(alpha_sv))])

    # Calculating the weight vector w
    w = np.sum(alpha_sv[:, np.newaxis] * sv_y[:, np.newaxis] * sv_x, axis=0)

    return w, b

# Regularization parameter
C = 1.0

# Solving for w and b
w, b = soft_margin_svm_dual(X, y, C)
# Print the optimal w and b
print("Optimal w:", w)
print("Optimal b:", b)


Optimal w: [-1.00209655  3.64671035]
Optimal b: -1.9006552904024685
